# Import modules

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

import os, sys
sys.path.insert(0, os.getcwd()+"/../../common")
from FileUtil import gpfs2WinPath

sys.path.insert(0, os.getcwd()+"/..")
from SEMContour import *
from MxpJob import MxpJob
from MxpStage import MxpStageXmlParser

# MXP Job outxml editing

## DL SEM EUV model data prepare, tigher D2DB KPI spec

In [2]:
jobpath = r'/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/GF_demo/Case02/3Tmp/CSEMModelCal/case23738/E19.02/baseline'
outxml = r'/gpfs2/scratch/peyang/jobs/GF_demo_case02_dataprepare/h/data/dummydb/MXP/job1/d2dbalignmentkpi550out.xml'
job = MxpJob(jobpath)
df = job.getStageResultFactory(enable=550, result_option='occfs')
print(df.shape)
print((df['costwt']>0).sum())

(df.filter(regex='^kpi/\w+_[x|y]{1}$').abs()<=1).head() #.all(axis=1).shape

d2dbkpis = [(df.filter(regex='^kpi/\w+_[x|y]{1}$').abs()<=spec).all(axis=1) for spec in [1, 1.5, 1.75, 2]]
print([kpi.sum() for kpi in d2dbkpis])
df.loc[~d2dbkpis[3], 'costwt'] = 0
MxpStageXmlParser.save(df, path=outxml)
df = df.set_index('name')

(583, 19)
241
[46, 136, 197, 241]


## DL SEM EUV model data prepare, 1D 2D ratio

In [57]:
dfs = []
df1d = df.select(lambda x: any([s in x for s in ['Bar', 'LS']]))
print("1D:", len(df1d), (df1d.filter(regex='^kpi/\w+_[x|y]{1}$').abs()<=2).all(axis=1).sum())
#dfs.append(df1d.loc[(df1d.filter(regex='^kpi/\w+_[x|y]{1}$').abs()<=2).all(axis=1), :]

df2d = df.select(lambda x: all([s not in x for s in ['Bar', 'LS']]))
print("2D:", len(df2d), (df2d.filter(regex='^kpi/\w+_[x|y]{1}$').abs()<=2).all(axis=1).sum())


1D: 205 84
2D: 378 157


In [6]:
from XmlUtil import dfFromConfigMapList
import xml.etree.ElementTree as ET
ocf = ET.parse(outxml).getroot().find("result")
df = dfFromConfigMapList(ocf, ".pattern")
len(df.loc[df.costwt>0, :])

241